In [12]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk
from itk import TubeTK as ttk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [17]:
def extrude_column(arr, col, val):
    col = int(col)
    for z in range(arr.shape[0]):
        pos = np.argmax(arr[z,:,col]==val)
        if pos>0:
            first = pos
            arr[z,:first,col]=0
            arr[z,first:,col]=val
    return arr
    
def extrude_annotations(pathname):
    data_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.nii.gz"))
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.interpolated-overlay.nii.gz"))
    
    num_overlays = len(data_overlay_paths)
    print(len(data_paths), num_overlays)
    for i in range(num_overlays):
        itkimg = itk.imread(data_overlay_paths[i], itk.UC)
        print("Processing", data_overlay_paths[i])
        
        ovrarr = itk.GetArrayFromImage(itkimg)
        
        nos = 0
        rib = 0
        sli = 0
        for c in range(ovrarr.shape[2]):
            nos = np.count_nonzero(ovrarr[:,:,c]==1)
            rib = np.count_nonzero(ovrarr[:,:,c]==2)
            sli = np.count_nonzero(ovrarr[:,:,c]==3)
            if nos>0 and rib==0:
                ovrarr = extrude_column(ovrarr,c,1)
            elif nos==0 and rib>0 and sli==0:
                ovrarr = extrude_column(ovrarr,c,2)
            elif sli>0 and rib==0:
                ovrarr = extrude_column(ovrarr,c,3)
            else:
                ovrarr[:,:,c]=0
                
        img = itk.imread(data_paths[i], itk.F)
        imMath = ttk.ImageMath.New(img)
        imMath.Blur(5)
        imMath.Threshold(0,40,0,1)
        imMath.Dilate(10,1,0)
        img = imMath.GetOutputUChar()
        connComp = ttk.SegmentConnectedComponents.New(img)
        connComp.SetMinimumVolume(100000)
        connComp.Update()
        arr_mask = itk.GetArrayFromImage(connComp.GetOutput())
        arr_mask = np.where(arr_mask>0,1,0)
        keep = arr_mask * ovrarr
        #keep = np.where(keep==0,2,keep)
        for val in [1,2,3]:
            ovrarr = np.where(ovrarr==val,keep,ovrarr)
        (filepath,filename) = os.path.split(data_overlay_paths[i])
        new_name = filepath+"/"+filename.split('.')[0]+'.extruded-overlay-NRS.nii.gz'
        print("Writing", new_name)
        itkimg = itk.GetImageFromArray(ovrarr.astype(np.ubyte))
        itk.imwrite(itkimg, new_name, compression=True)

In [19]:
pathname = "../../Data/VFoldData/BAMC-PTXNoSliding"
extrude_annotations(pathname)
pathname = "../../Data/VFoldData/BAMC-PTXSliding"
extrude_annotations(pathname)

30 30
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_Image_262499828648_clean.interpolated-overlay.nii.gz
Writing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_Image_262499828648_clean.extruded-overlay-NRS.nii.gz
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_image_267456908021_clean.interpolated-overlay.nii.gz
Writing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_image_267456908021_clean.extruded-overlay-NRS.nii.gz
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1083297968960_clean.interpolated-overlay.nii.gz
Writing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1083297968960_clean.extruded-overlay-NRS.nii.gz
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1087766719219_clean.interpolated-overlay.nii.gz
Writing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1087766719219_clean.extrud

Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10705997566592_CLEAN.extruded-overlay-NRS.nii.gz
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10891015221417_clean.interpolated-overlay.nii.gz
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10891015221417_clean.extruded-overlay-NRS.nii.gz
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/030s_iimage_1180496934444_clean.interpolated-overlay.nii.gz
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/030s_iimage_1180496934444_clean.extruded-overlay-NRS.nii.gz
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/030s_iimage_677741729740_clean.interpolated-overlay.nii.gz
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/030s_iimage_677741729740_clean.extruded-overlay-NRS.nii.gz
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/034s_iimage_3368391807672_clean.interpolated-overlay.ni

In [ ]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
extrude_annotations(pathname)

In [73]:
pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
extrude_annotations(pathname)
pathname = "../../Data/TestingData/BAMC-PTXSliding"
extrude_annotations(pathname)

Processing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear\image_417221672548_CLEAN.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_417221672548_CLEAN.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear\image_426794579576_CLEAN.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_426794579576_CLEAN.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear\image_3925135436261_clean.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear/image_3925135436261_clean.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear\image_3929217595322_clean.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear/image_3929217595322_clean.extruded-overlay.mha
